In [6]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)


2022-02-11 17:22:49 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75


In [15]:
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(models, cfg)

text = "The following system shown below in Figure 3 is used for imaging tissues and measuring fluorescence from it in endoscopy. The scanning fiber core diameter is ~ 5 mm while the collection fibers core diameter is 200 mm. The lens assembly creates an imaging system (M=1, NA =0.075, F_obj = 1mm) to image the scanning fiber spot into the tissue surface, located 1 mm away from the tip of the collection fibers."

model = model.to("cuda:0")

sample = TTSHubInterface.get_model_input(task, text)
sample["net_input"]["src_tokens"] = sample["net_input"]["src_tokens"].to("cuda:0")
sample["net_input"]["src_lengths"] = sample["net_input"]["src_lengths"].to("cuda:0")
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)


2022-02-11 17:31:53 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from C:/Users/runze/.cache/fairseq/facebook--fastspeech2-en-ljspeech.main.a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin


In [16]:
import numpy as np
from scipy.io.wavfile import write

data = wav.cpu().numpy() # 44100 random samples between -1 and 1
scaled = np.int16(data/np.max(np.abs(data)) * 32767)
write('test.wav', rate, scaled)